In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

education_df = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()

In [3]:
#Using interpolation since data is time series data (helps with continuity, use with ordered data)

col_list = ['ENROLL', 'TOTAL_REVENUE','FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE', 'TOTAL_EXPENDITURE',
           'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE',
           'GRADES_PK_G', 'GRADES_KG_G', 'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G', 'GRADES_9_12_G',
           'GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', 'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE']

for col in col_list:
    education_df.loc[:,col].fillna(education_df.loc[:,col].interpolate(),inplace=True)
    
education_df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,211.790904,258.776900,207.088116,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,265.519676


## We want to understand the relationship between the expenditures of the governments and the students' overall success in math and reading.

### 1. Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores.

In [4]:
education_df['OVERALL_SCORE'] = (((education_df.AVG_MATH_4_SCORE)*0.5 + (education_df.AVG_READING_4_SCORE)*0.5)) + (((education_df.AVG_MATH_8_SCORE)*0.5 + (education_df.AVG_READING_8_SCORE)*0.5))/(education_df.GRADES_4_G + education_df.GRADES_8_G)
education_df.drop(education_df.index[0], inplace=True)

### 2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [5]:
education_df[['OVERALL_SCORE', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
              'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']].corr()

,OVERALL_SCORE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
OVERALL_SCORE,1.000000,0.252573,0.255551,0.274762,0.211470,0.168228
TOTAL_EXPENDITURE,0.252573,1.000000,0.992755,0.992494,0.951726,0.928421
INSTRUCTION_EXPENDITURE,0.255551,0.992755,1.000000,0.979321,0.920297,0.895994
SUPPORT_SERVICES_EXPENDITURE,0.274762,0.992494,0.979321,1.000000,0.953411,0.905755
OTHER_EXPENDITURE,0.211470,0.951726,0.920297,0.953411,1.000000,0.923468
CAPITAL_OUTLAY_EXPENDITURE,0.168228,0.928421,0.895994,0.905755,0.923468,1.000000


The support services expenditure is the most correlated variable with overall score. 

### 3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [6]:
education_df.dropna(inplace=True)

X = education_df[['INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE',
                  'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
education_df['PCA_1'] = sklearn_pca.fit_transform(X)

print('The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.94725496]


94% of the variance is explained by the first component.  

### 4. What is the correlation between the overall score variable and the 1st principal component?

In [7]:
education_df[['OVERALL_SCORE', 'PCA_1','TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
              'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']].corr()

,OVERALL_SCORE,PCA_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
OVERALL_SCORE,1.000000,0.230972,0.249063,0.252588,0.272157,0.211470,0.161267
PCA_1,0.230972,1.000000,0.992988,0.975096,0.986139,0.975451,0.956156
TOTAL_EXPENDITURE,0.249063,0.992988,1.000000,0.992698,0.992435,0.951726,0.928129
INSTRUCTION_EXPENDITURE,0.252588,0.975096,0.992698,1.000000,0.979165,0.920297,0.895527
SUPPORT_SERVICES_EXPENDITURE,0.272157,0.986139,0.992435,0.979165,1.000000,0.953411,0.905265
OTHER_EXPENDITURE,0.211470,0.975451,0.951726,0.920297,0.953411,1.000000,0.923468
CAPITAL_OUTLAY_EXPENDITURE,0.161267,0.956156,0.928129,0.895527,0.905265,0.923468,1.000000


The correlation between the overall score and the first principal component is 23%.  The first principal component is over 95% correlated to all four of the expenditure types.  

### 5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

I would use the first principal component because it is similarly correlated to the overall score as the expenditure types (~2% difference).  In addition, the PCA_1 and all expenditure types are over 95% correlated, so there is minimal variance missing from the four expenditure types and PCA_1.  